In [1]:
import numpy as np
import pydot
from IPython.display import HTML, SVG, display
from pydrake.all import (
    AddMultibodyPlantSceneGraph,
    DiagramBuilder,
    GenerateHtml,
    InverseDynamicsController,
    PidController,
    MeshcatVisualizer,
    MeshcatVisualizerParams,
    MultibodyPlant,
    Parser,
    Simulator,
    StartMeshcat,
    SceneGraph,
    PassThrough,
    Demultiplexer,
    LeafSystem,
)

from catbot.utils.meshcat_util import MeshcatCatBotSliders 

In [2]:
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7001


In [7]:
# -- Add Reward (for testing RL Reward) -- #
class RewardSystem(LeafSystem):
    def __init__(self):
        LeafSystem.__init__(self)
        self.DeclareVectorInputPort("catbot_state", 10)
        self.DeclareVectorOutputPort("reward", 1, self.CalcReward)

    def CalcReward(self, context, output):
        # state is Center, a_hinge, a_rot, b_hinge, b_rot
        catbot_state = self.get_input_port(0).Eval(context)
        print('center: ', catbot_state[0])
        print('a_rot: ', catbot_state[2])
        print('b_rot: ', catbot_state[4])

        # So we clamp the angle between 0 and 2pi
        a_rot_world = (catbot_state[0] + catbot_state[2]) % (np.pi * 2) - np.pi
        b_rot_world = (catbot_state[0] + catbot_state[4]) % (np.pi * 2) - np.pi

        a_hinge = catbot_state[1]
        b_hinge = catbot_state[3]
        print('a_hinge: ', a_hinge)
        print('b_hinge: ', b_hinge)

        center_from_vertical = (catbot_state[0] % (2 * np.pi)) - np.pi

        print('a_world: ', a_rot_world)
        print('b_world: ', b_rot_world)


        # Add position cost
        # cost = a_hinge_world **2 + b_hinge_world **2
        # cost = center_from_vertical ** 2
        
        # cost = -a_rot_world * b_rot_world + center_from_vertical**2
        # cost = (a_rot_world**2 + b_rot_world**2) + 10 * np.sign(-a_rot_world * b_rot_world)
        # print('sign: ', np.sign(-a_rot_world * b_rot_world))

        # Add cost for the hinges being the same (symmetric motion)
        cost = a_rot_world**2 + b_rot_world**2 + 10 * (a_hinge - b_hinge)**2
        if a_rot_world ** 2 < (np.pi / 8)**2 and b_rot_world ** 2 < (np.pi / 8)**2:
            cost -= 100


        state_to_control_projection = np.array([
            [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
            [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
        ])

        # Note that we don't include effort cost here

        print('cost: ', cost)
        output[0] = 20 - cost

In [8]:
time_step = 1e-3

builder = DiagramBuilder()

# -- Add original plant -- #
# Adds both MultibodyPlant and the SceneGraph, and wires them together.
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=time_step)
# Note that we parse into both the plant and the scene_graph here.
model = Parser(plant, scene_graph).AddModelFromFile(
    "../models/singleAxisCatBot.urdf")

gravity_field = plant.mutable_gravity_field()
gravity_field.set_gravity_vector(np.array([0.0, 0.0, 0.0]))

plant.Finalize()

# print('plant names: ', plant.GetPositionNames(model))
# print('plant state: ', plant.GetState(model))

# -- Add controller plant -- #
controller_plant = MultibodyPlant(time_step=time_step)
model = Parser(controller_plant).AddModelFromFile(
    "../models/singleAxisCatBot.urdf")
controller_gravity_field = controller_plant.mutable_gravity_field()
controller_gravity_field.set_gravity_vector(np.array([0.0, 0.0, 0.0]))
controller_plant.Finalize()

# -- Add visualizer -- #
visualizer = MeshcatVisualizer.AddToBuilder(builder, 
                                            scene_graph.get_query_output_port(),
                                            meshcat)

meshcat.ResetRenderMode()
meshcat.DeleteAddedControls()

# -- Add controller -- #
# Create a PID controller for each joint.
num_model_actuators = controller_plant.num_actuators()

# state_projection_matrix = np.array([
#     [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
#     [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
#     [0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
#     [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],

#     [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
#     [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
#     [0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
#     [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
# ])

# actuator pos is a_rev, b_rev, a_hinge, b_hinge
# state is Center, a_hinge?, a_rot, b_hinge, b_rot 
# idk why they're different
state_projection_matrix = np.array([
    [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
    [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 0, 0, 0, 0, 0],

    [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
    [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
])

# kp = [0.0000, 0.000, 0.02, 0.02]
# ki = [0.00, 0.00, 0.0, 0.0]
# kd = [0.00, 0.00, 0.02, 0.02]

kp = [0.03, 0.03, 0.03, 0.03]
ki = [0.00, 0.00, 0.0, 0.0]
kd = [0.025, 0.025, 0.04, 0.04]
catbot_controller = builder.AddSystem(
    PidController(state_projection_matrix, kp, ki, kd)
)

catbot_controller.set_name("catbot_controller")
builder.Connect(
    plant.get_state_output_port(model),
    catbot_controller.get_input_port_estimated_state(),
)
builder.Connect(
    catbot_controller.get_output_port_control(), 
    plant.get_actuation_input_port()
)

# -- Set up teleop wigits -- #
teleop = builder.AddSystem(
    MeshcatCatBotSliders(
        meshcat,
    ))

builder.Connect(teleop.get_output_port(0), catbot_controller.get_input_port_desired_state())
builder.Connect(plant.get_state_output_port(), teleop.get_input_port(0))

# -- Add reward system -- #
reward = builder.AddSystem(RewardSystem())
builder.Connect(plant.get_state_output_port(), reward.get_input_port(0))
builder.Connect(reward.get_output_port(0), teleop.get_input_port(1))


diagram = builder.Build()
simulator = Simulator(diagram)
context = simulator.get_mutable_context()

simulator.set_target_realtime_rate(1.0)
meshcat.AddButton("Stop Simulation", "Escape")
print("Press Escape to stop the simulation")

# -- Set init pose -- #
# state is Center, a_hinge?, a_rot, b_hinge, b_rot 
q0 = [0.0, 0.00, 0.00, 0.0, 0.0]
plant_context = plant.GetMyMutableContextFromRoot(context)
plant.SetPositions(plant_context, q0)
simulator.AdvanceTo(0.01)

# -- Uncomment to simulate -- #
cnt = 0
while meshcat.GetButtonClicks("Stop Simulation") < 1:
    simulator.AdvanceTo(simulator.get_context().get_time() + 1.0)
    # if cnt % 10 == 0:
    #     print_status()
    #     input('Continue')
    # cnt += 1

meshcat.DeleteButton("Stop Simulation")

Keyboard Controls:
a_rev : KeyQ / KeyE
b_rev : KeyW / KeyS
a_hinge : KeyA / KeyD
b_hinge : KeyJ / KeyL
Press Escape to stop the simulation
Pos:  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
center:  0.0
a_rot:  0.0
b_rot:  0.0
a_hinge:  0.0
b_hinge:  0.0
a_world:  -3.141592653589793
b_world:  -3.141592653589793
cost:  19.739208802178716
Reward: [0.2607912]
center:  0.0
a_rot:  0.0
b_rot:  0.0
a_hinge:  0.07960647834128735
b_hinge:  0.0
a_world:  -3.141592653589793
b_world:  -3.141592653589793
cost:  19.802580716117735
Reward: [0.19741928]
center:  0.0
a_rot:  0.0
b_rot:  0.0
a_hinge:  0.33820567871791635
b_hinge:  0.3389211709675058
a_world:  -3.141592653589793
b_world:  -3.141592653589793
cost:  19.73921392147031
Reward: [0.26078608]
center:  0.028551024467693448
a_rot:  -0.00010633141943787474
b_rot:  -0.22789724277298642
a_hinge:  0.3491684012716213
b_hinge:  0.35948520067130413
a_world:  -3.1131479605415375
b_world:  2.9422464352845
cost:  18.349568673666813
Reward: [1.65043133]
center:  0.0335